In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("../data/processed/cleaned_data_after_eda.csv")
df.head()
df = df.drop(columns = ["Yatırıma_Uygunluk","Takas","Eşya_Durumu"])

In [2]:
df["Fiyat"] = np.expm1(df["Fiyat"])

# df = df.drop(columns = [""])
condition_to_remove1 = (df["Net_Metrekare"] > 300) & (df["Fiyat"] < 2500000)
condition_to_remove2 = (df["Net_Metrekare"] < 100) & (df["Fiyat"] > 10000000)
condition_to_remove3 = (df["Fiyat"] < 500000)

cond_cheap_large_house = (df['Oda_Sayısı'] >= 5) & (df["Fiyat"] < 1500000)
cond_expensive_small_house = (df['Oda_Sayısı'] <= 2) & (df["Fiyat"] > 8000000)
cond_absolute_floor = (df["Fiyat"] < 500000)

noise_filter1 = condition_to_remove1 | condition_to_remove2 | condition_to_remove3
room_noise = cond_cheap_large_house | cond_expensive_small_house | cond_absolute_floor
df = df[~noise_filter1]
df = df[~room_noise]
# for col in numerical_columns:

#     df.boxplot(column = col)
#     plt.show()



C:\Users\karac\AppData\Local\Temp\ipykernel_1152\2051741078.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~room_noise]


## CatBoost Model Training 

In [3]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

df["Fiyat"] = np.log1p(df["Fiyat"])
y1 = df["Fiyat"]
x1 = df.drop("Fiyat", axis=1)
cat_columns = np.where(x1.dtypes != float)[0]

x_train1, x_test1, y_train1, y_test1 = train_test_split(x1,y1,test_size=0.2,random_state=34,stratify=None)


model_pipeline1 = Pipeline(steps =[("model",CatBoostRegressor(cat_features=cat_columns,loss_function='RMSE', verbose=0,random_state=42, thread_count=-1))])
log_model = TransformedTargetRegressor(
    regressor=model_pipeline1,
    func=np.log1p,
    inverse_func=np.expm1
)

model_pipeline1.fit(x_train1, y_train1)

y_predict1 = model_pipeline1.predict(x_test1)



## CatBoost Model Evaluation

In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


rmse1 = np.sqrt(mean_squared_error(y_test1,y_predict1))
mae1 = mean_absolute_error(y_test1,y_predict1)
r21 = r2_score(y_test1,y_predict1)

train_score1 = model_pipeline1.score(x_train1,y_train1)
test_score1 = model_pipeline1.score(x_test1,y_test1)

print(f"RMSE: {rmse1:.2f}") 
print(f"MAE:  {mae1:.2f}")
print(f"R2:   {r21:.2f}")

print(f"Training R2: %{train_score1*100:.2f}")
print(f"Testing R2:  %{test_score1*100:.2f}")



RMSE: 0.29
MAE:  0.21
R2:   0.68
Training R2: %74.31
Testing R2:  %67.51


## Pickling the Catboost Model

In [6]:
#import pickle
#with open("../Model/best_model.pkl","wb") as f:
#    pickle.dump(model_pipeline1,f)

## Data Encoding for The Random Forest Model

In [7]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import TargetEncoder

# y2 = df["Fiyat"]
# x2 = df.drop("Fiyat", axis=1)


# x_train2, x_test2, y_train2, y_test2 = train_test_split(x2,y2,test_size=0.2,random_state=65,stratify=None)

# cat_columns= (df.select_dtypes(include=["object"])).columns

# ordinal_cols = [col for col in cat_columns if col != "Şehir"]


# rf_ct = ColumnTransformer(
#     transformers=[
#     ('target_enc', TargetEncoder(), ["Şehir"]),
#     ("encoder",OrdinalEncoder(),ordinal_cols)
#     ],
#     remainder="passthrough"
# )

# model_pipeline2 = Pipeline(steps=[
#     ("transformer", rf_ct),
#     ("model",RandomForestRegressor(random_state=34))
# ])


# model_pipeline2.fit(x_train2,y_train2)
# y_predict2 =model_pipeline2.predict(x_test2)






## Random Forest Model Evaluation

In [8]:
# rmse2 = np.sqrt(mean_squared_error(y_test2,y_predict2))
# mae2 = mean_absolute_error(y_test2,y_predict2)
# r22 = r2_score(y_test2,y_predict2)

# train_score2 = model_pipeline2.score(x_train2,y_train2)
# test_score2 = model_pipeline2.score(x_test2,y_test2)

# print(f"RMSE: %{rmse2*100:.4f}") 
# print(f"MAE:  %{mae2*100:.4f}")
# print(f"R2:   %{r22*100:.4f}")

# print(f"Training R2: %{train_score2*100:.4f}")
# print(f"Testing R2:  %{test_score2*100:.4f}")